This is my toxicBert For classification. I collpase hate speech and offensive language to one class as toxic language

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/drive/MyDrive/ToxicTrainingData/training.csv', header='infer' )
df['id'] = df.index
df = df.dropna()
df['toxic'] = df['toxic'].astype('int32')


In [ ]:
display(df)

,text,toxic,id
0,well how else will white ppl get us to forg...,1,0
1,Funny thing is....it's not just the people doi...,0,1
2,""" "" Nigga messed with the wrong bitch #12855...",1,2
3,bitch ass nigggaaa,1,3
4,S/o that real bitch,1,4
...,...,...,...
65217,when the people you love the most have to let ...,0,65217
65218,Fuck niggah add it up..,1,65218
65219,her pussy stay wet when shes around me gotta...,1,65219
65220,Let a bitch be a bitch n a hoe be a hoe!,1,65220


In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
train_val_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_val_split['train']  # 90% of the data
val_dataset = train_val_split['test']  # 10% of the data

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

In [ ]:
def tokenize_function(examples):
    # Tokenize the text
    tokenized_examples = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_attention_mask=True
    )
    # Add labels
    tokenized_examples["labels"] = examples["toxic"]
    return tokenized_examples

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/58699 [00:00<?, ? examples/s]

Map:   0%|          | 0/6523 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/result",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/logs",
    evaluation_strategy="steps",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets
)


trainer.train()

Step,Training Loss,Validation Loss
10,1.042100,0.794790
20,0.628700,0.599322
30,0.665900,0.423747
40,0.326500,0.323814
50,0.261800,0.292814
60,0.255700,0.236366
70,0.270000,0.216144
80,0.241900,0.195835
90,0.170000,0.180972
100,0.099100,0.215699


Step,Training Loss,Validation Loss
10,1.042100,0.794790
20,0.628700,0.599322
30,0.665900,0.423747
40,0.326500,0.323814
50,0.261800,0.292814
60,0.255700,0.236366
70,0.270000,0.216144
80,0.241900,0.195835
90,0.170000,0.180972
100,0.099100,0.215699


In [ ]:
model.save_pretrained("/content/drive/MyDrive/ToxicBert")

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# test and reload the model

In [ ]:
import torch
from torch.nn.functional import softmax

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "/content/drive/MyDrive/ToxicBert"

# Load tokenizer from a specific path or identifier
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

# Load model from the directory where your 'config.json' and model weights are saved
model = AutoModelForSequenceClassification.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
print (model.config)

RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/ToxicBert",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NOT-HATE",
    "1": "HATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "HATE": 1,
    "NOT-HATE": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [ ]:
def tokenize_function(examples):
    # Tokenize the text
    tokenized_examples = tokenizer(
        examples,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt"  # Ensure output is PyTorch tensors
    )
    return tokenized_examples

In [ ]:
# Predict
inputs = tokenize_function ("""mage. why? portals sheep motha fuckin lasers

BLOW SHIT UP WITH FIRE YO

ah shit son? is that something comin at ya? NAW NIGGA, I GOT DAT FROSTBOLT.

You feelin lonely? MIRROR IMAGE NIGGUH. YOU GOT 3 FRIENDS NOW.

aw damn son u still dont feel powerful? WELL U CAN CONTROL MOTHA FUCKIN TIME SON

MOTHA.
 FUCKING.
TIME. """)

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits

# # Apply softmax to logits to get probabilities
# probs = softmax(logits, dim=1)

# Get the predicted class ID and probability
predicted_class_id = logits.argmax().item()
# predicted_prob = probs.max().item()

# # Map the predicted class ID to its label
predicted_label = model.config.id2label[int(predicted_class_id)]

print(f"Predicted Label: {predicted_label}).")

Predicted Label: HATE).


In [ ]:
def predict (tokenized_text):
    with torch.no_grad():
        outputs = model(**tokenized_text)

        logits = outputs.logits

        predicted_class_id = logits.argmax().item()

        #predicted_label = model.config.id2label[int(predicted_class_id)]

    return predicted_class_id


In [ ]:
text2 ="""All for some oil contracts from Libya eh? It's times like black these I wish there were a hell so these people could rot there for all of eternity.

Thanks to Wikileaks for showing us just fuch, shit , gay what our governments will do to suck on that huge oil dick.  """
token_text1 =  tokenize_function(text2)
lablel = predict(token_text1)
print (lablel)
predicted_label = model.config.id2label[int(lablel)]
print (predicted_label)


0
NOT-HATE


In [ ]:
text1 ="""
You are not the same person as me.  I enjoy beating good players just as much as everyone else, and i'm not going to quit when i'm outplayed. I'm going to quit almost immediately when I'm playing a "good team"

 When I say "good team", I actually mean those faggot kids that run ghost, SOH/warlord, ninja, and soundwhore everyone with the turt13 b34c|-|35 their mom bought them while using FAMAS/74u w/claymore and claiming that they're "good" at the game.  With this combination, killstreaks are nearly impossible, and when attained they are actually completely useless.

This game is inherently imbalanced by nature, as is every CoD, and this is why there isn't a legitimate (read: IRL MLG, not gamebattles bullshit) competitive scene for it for the most part, in contrast to Halo, Gears of War, CS, Quake, or Starcraft 2.  I'm not being a pussy by leaving, I'm not wasting my time getting 20 deaths from the same faggot kid's gunship camping my spawn on Nuketown.  This is an arcade shooter, not a "tactical" or "skilled" shooter.  It's played for fun, not competition.  I'm not berating CoD as a series, but everything has a fatal flaw; this is CoD's.  Gears and Halo don't suffer this particular problem because you can't spawn with imba guns, and the imba guns are super imba and you have to actually best players with shitty guns before you can get the good guns (creating balance).  In CS you can only get good equipment for playing well, and as soon as you are bested by somebody else you LOSE ALL OF THAT EQUIPMENT.  I've never played Quake so I can't comment on that, however I promise that you can't get a FAMAS in quake.  In SC2 there are only 3 choices of race, and each is dependent entirely on how well you play over an extended period of time to win as opposed to .84 seconds of FAMAS spray that you may or may not land due to the bullshit netcode that Treyarch decided to change from MW2.

If you want to play a video game for competitive purposes, I dare you to get on SC2, or even SCBW.  GLHF in bronze/D-.  When you don't do this, you can see me with a team of redditors skullfucking pubtards all day long, because that's what I find fun.  If we find competition, all we've gotta do is bump down the killstreaks and continue domination until we hit one of my aforementioned "good teams", and by that point we just laugh at them for being stupid and homosexual and find a new room.

I guess that makes me a shitty video game player.  I can live with that.

I'd also like to make it known that I respect and appreciate you and your opinion. I'm not trying to say "you're a fucking retard" or anything like that, pwomise &lt;3

EDIT:  On a related note OMFUX I CAN'T WAIT FOR HOMEFRONT.  It sounds absolutely wonderful, especially with it's BattlePoints system :)
"""
token_text2 =  tokenize_function(text1)
label = predict(token_text2)
print (label)
predicted_label = model.config.id2label[int(label)]
print (predicted_label)

1
HATE


In [ ]:
text1 ="""

Because you're all a bunch of racist bastards with centuries old bullshit issues with every other nation surrounding you.
"""
token_text2 =  tokenize_function(text1)
label = predict(token_text2)
print (label)
print (type(label))
predicted_label = model.config.id2label[int(label)]
print (predicted_label)

1
<class 'int'>
HATE


In [ ]:
### this function will do the prediction
###  1 for Toxic (True)
###  0 for Not_Toxic (False)

def predict (tokenized_text):
    with torch.no_grad():
        outputs = model(**tokenized_text)

        logits = outputs.logits

        predicted_class_id = logits.argmax().item()
        return predicted_class_id

In [ ]:
### load test dataset
texts = []
labels = []
with open('/content/test_text.txt') as f:
    lines = f.readlines()
    print (len(lines))
    texts = (lines)
with open('/content/test_labels.txt') as f:
    lines = f.readlines()
    print (len(lines))
    labels = (lines)
print (len(texts), len(labels))

2970
2970
2970 2970


In [ ]:
predict_label = [None for i in range (len(labels))]
print(len(predict_label))

2970


In [ ]:
for i in range (len(texts)):
    texts[i] =  tokenize_function(texts[i])

In [ ]:
for i in range (len(texts)):
    predict_label[i] =  predict(texts[i])

In [ ]:
print (predict_label[5])

0


In [ ]:
labels = [int(labels[i]) for i in range (len(labels))]

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

true_labels = labels
# Calculate metrics
f1 = f1_score(true_labels, predict_label)
recall = recall_score(true_labels, predict_label)
precision = precision_score(true_labels, predict_label)
accuracy = accuracy_score(true_labels, predict_label)

# Print metrics
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")

F1 Score: 0.5113752122241088
Recall: 0.6014376996805112
Precision: 0.44477259303012406
Accuracy: 0.5154882154882154


In [ ]:
import pandas as pd
file_paths = ["/content/drive/MyDrive/ToxicTrainingData/test1.csv","/content/drive/MyDrive/ToxicTrainingData/test2.csv"]
texts = []
labels = []
for path in file_paths:
    df = pd.read_csv(path, header='infer' )
    texts.extend (df['text'].tolist())
    labels.extend (df['toxic'].tolist())
    del df
predict_labels = [None for i in range( len(labels))]
print (len(texts),len(labels), len(predict_labels))

16306 16306 16306


In [ ]:
print (type(labels[1]))
texts = [ tokenize_function(texts[i]) for i in range (len(texts))    ]
predict_labels =  [ predict(texts[i]) for i in range (len(texts))]


<class 'int'>


In [ ]:
true_labels = labels
# Calculate metrics
f1 = f1_score(true_labels, predict_labels)
recall = recall_score(true_labels, predict_labels)
precision = precision_score(true_labels, predict_labels)
accuracy = accuracy_score(true_labels, predict_labels)

# Print metrics
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")

F1 Score: 0.9774878780882014
Recall: 0.9764733018106332
Precision: 0.9785045648907893
Accuracy: 0.976082423647737


# compare with the orginal model

In [ ]:
# load the orgianl model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

In [ ]:
def tokenize_function(examples):
    # Tokenize the text
    tokenized_examples = tokenizer(
        examples,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_attention_mask=True
    )
    return tokenized_examples